### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [162]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
print(student_data["student_name"].nunique())
print(school_data.head())
#students = student_data["student_name"].value_counts()
#print(students.head())
#mikes = student_data.loc[(student_data["student_name"]=="Michael Smith")]
#print (mikes)
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#rename df
comp_data_df = school_data_complete
comp_data_df.head()

32715
   School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [163]:
tot_school_cnt = comp_data_df["school_name"].nunique()
#print("tot # schools:"+str(tot_school_cnt))
tot_student_cnt = comp_data_df["Student ID"].nunique()   # some students have same name, so count the unique student IDs.
tot_budget = school_data["budget"].sum()
avg_math = student_data["math_score"].mean()
avg_reading = student_data["reading_score"].mean()
# % students passed (70% or greater = passing) 
num_passed_math = len(student_data.loc[(student_data["math_score"]>=70)])
pct_passed_math = (num_passed_math/tot_student_cnt)*100
num_passed_reading = len(student_data.loc[(student_data["reading_score"]>=70)])
pct_passed_reading = (num_passed_reading/tot_student_cnt)*100

num_passed_both = len(student_data.loc[((student_data["math_score"]>=70) & (student_data["reading_score"]>=70))])
pct_passed_both = (num_passed_both/tot_student_cnt)*100

summary_df = pd.DataFrame({"Total Schools": [tot_school_cnt],
                           "Total Students": [tot_student_cnt],
                           "Total Budget": [tot_budget],
                           "Average Math Score": avg_math,
                           "Average Reading Score": avg_reading,
                           "% Passing Math": pct_passed_math,
                          "% Passing Reading": pct_passed_reading,
                          "% Overall Passing": pct_passed_both})
summary_df.style.format({"Total Students":"{:,}","Total Budget":"${:,.2f}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [164]:
school_groupby = student_data.groupby("school_name")
#school_groupby["Student ID"].count()
stype = school_data[["school_name","type","budget"]]
#print(school_data)
#print(stype.dtypes)

school_summ_df = comp_data_df["school_name"].value_counts().to_frame().reset_index().rename(columns = {"index":"school_name","school_name":"Total Students"})

#type(school_data[["school_name","type"]])#school_summ_df)
school_summ_df = pd.merge(school_summ_df, stype, how="left", on=["school_name", "school_name"]).rename(columns = {"type":"School Type","budget":"Total Budget"})
school_summ_df = school_summ_df.sort_values(by=["school_name"])
school_summ_df["Per Student Budget"] =school_summ_df["Total Budget"]/school_summ_df["Total Students"] 
school_summ_df


,school_name,Total Students,School Type,Total Budget,Per Student Budget
0,Bailey High School,4976,District,3124928,628.0
8,Cabrera High School,1858,Charter,1081356,582.0
4,Figueroa High School,2949,District,1884411,639.0
6,Ford High School,2739,District,1763916,644.0
12,Griffin High School,1468,Charter,917500,625.0
2,Hernandez High School,4635,District,3022020,652.0
14,Holden High School,427,Charter,248087,581.0
5,Huang High School,2917,District,1910635,655.0
1,Johnson High School,4761,District,3094650,650.0
13,Pena High School,962,Charter,585858,609.0


In [165]:
# create avgs and merge into summary
avgs_df = school_groupby.mean().reset_index().rename(columns = {"reading_score":"Avg Reading Score","math_score":"Avg Math Score"})
print(avgs_df)
del avgs_df["Student ID"]
school_summ_df = pd.merge(school_summ_df, avgs_df, how="left", on=["school_name", "school_name"])
school_summ_df

              school_name  Student ID  Avg Reading Score  Avg Math Score
0      Bailey High School     20358.5          81.033963       77.048432
1     Cabrera High School     16941.5          83.975780       83.061895
2    Figueroa High School      4391.0          81.158020       76.711767
3        Ford High School     36165.0          80.746258       77.102592
4     Griffin High School     12995.5          83.816757       83.351499
5   Hernandez High School      9944.0          80.934412       77.289752
6      Holden High School     23060.0          83.814988       83.803279
7       Huang High School      1458.0          81.182722       76.629414
8     Johnson High School     32415.0          80.966394       77.072464
9        Pena High School     23754.5          84.044699       83.839917
10  Rodriguez High School     28035.0          80.744686       76.842711
11    Shelton High School      6746.0          83.725724       83.359455
12     Thomas High School     38352.0          83.8

,school_name,Total Students,School Type,Total Budget,Per Student Budget,Avg Reading Score,Avg Math Score
0,Bailey High School,4976,District,3124928,628.0,81.033963,77.048432
1,Cabrera High School,1858,Charter,1081356,582.0,83.975780,83.061895
2,Figueroa High School,2949,District,1884411,639.0,81.158020,76.711767
3,Ford High School,2739,District,1763916,644.0,80.746258,77.102592
4,Griffin High School,1468,Charter,917500,625.0,83.816757,83.351499
5,Hernandez High School,4635,District,3022020,652.0,80.934412,77.289752
6,Holden High School,427,Charter,248087,581.0,83.814988,83.803279
7,Huang High School,2917,District,1910635,655.0,81.182722,76.629414
8,Johnson High School,4761,District,3094650,650.0,80.966394,77.072464
9,Pena High School,962,Charter,585858,609.0,84.044699,83.839917


In [195]:
# update the index to be the school name   (todo - double check above code if can remove the reset index...and then don't need this)
school_summ_df= school_summ_df.set_index("school_name")  


In [201]:
#add new columns, init to 0
school_summ_df["% Passing Math"]=0.000000   #6 decimal places to match output of example 
school_summ_df["% Passing Reading"]=0.000000  
school_summ_df["% Overall Passing"]=0.000000 

In [202]:
# figure out % passing per school for math/reading/both
student_data
#test_df = school_summ_df.set_index("school_name")   #this works

#print(school_summ_df.head())
test_df.index.values.tolist()
# test statement to see if can get the # of students passed math
#num_passed_math = len(student_data.loc[((student_data["school_name"]=="Bailey High School")&(student_data["math_score"]>=70))])
print((num_passed_math/4976)*100)
for school in test_df.index.values.tolist():  #todo  update this to be the real  df instead of test
    print("in loop "+school+" #students:" + str(school_summ_df.loc[school,"Total Students"]))
    ## calc math %
    num_passed_math = len(student_data.loc[((student_data["school_name"]==school)&(student_data["math_score"]>=70))])
    school_summ_df.at[school,"% Passing Math"]= (num_passed_math/school_summ_df.loc[school,"Total Students"])*100 
    ## calc reading %
    num_passed_reading = len(student_data.loc[((student_data["school_name"]==school)&(student_data["reading_score"]>=70))])
    school_summ_df.at[school,"% Passing Reading"]= (num_passed_reading/school_summ_df.loc[school,"Total Students"])*100 
    ## calc overall %
    num_passed_reading = len(student_data.loc[((student_data["school_name"]==school)&(student_data["math_score"]>=70)&(student_data["reading_score"]>=70))])
    school_summ_df.at[school,"% Overall Passing"]= (num_passed_reading/school_summ_df.loc[school,"Total Students"])*100 

33.762057877813504
in loop Bailey High School #students:4976
in loop Cabrera High School #students:1858
in loop Figueroa High School #students:2949
in loop Ford High School #students:2739
in loop Griffin High School #students:1468
in loop Hernandez High School #students:4635
in loop Holden High School #students:427
in loop Huang High School #students:2917
in loop Johnson High School #students:4761
in loop Pena High School #students:962
in loop Rodriguez High School #students:3999
in loop Shelton High School #students:1761
in loop Thomas High School #students:1635
in loop Wilson High School #students:2283
in loop Wright High School #students:1800


In [203]:
school_summ_df

,Total Students,School Type,Total Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,4976,District,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
Cabrera High School,1858,Charter,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
Figueroa High School,2949,District,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
Ford High School,2739,District,1763916,644.0,80.746258,77.102592,68.309602,79.299014,54.289887
Griffin High School,1468,Charter,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
Hernandez High School,4635,District,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
Holden High School,427,Charter,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166
Huang High School,2917,District,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
Johnson High School,4761,District,3094650,650.0,80.966394,77.072464,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [206]:
#print(school_summ_df.sort_values(by=["% Overall Passing"] ascending = False).head(5)  #could use print, or just list what you want the notebook to print like below statement.   below statement looks nicer
school_summ_df.sort_values(by=["% Overall Passing"], ascending = False).head(5)

,Total Students,School Type,Total Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,1858,Charter,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
Thomas High School,1635,Charter,1043130,638.0,83.848930,83.418349,93.272171,97.308869,90.948012
Griffin High School,1468,Charter,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
Wilson High School,2283,Charter,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
Pena High School,962,Charter,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [207]:
#print(school_summ_df.sort_values(by=["% Overall Passing"]).head(5))  #could use print, or just list what you want the notebook to print like below statement.   below statement looks nicer
school_summ_df.sort_values(by=["% Overall Passing"]).head(5)

,Total Students,School Type,Total Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,3999,District,2547363,637.0,80.744686,76.842711,66.366592,80.220055,52.988247
Figueroa High School,2949,District,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
Huang High School,2917,District,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
Hernandez High School,4635,District,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
Johnson High School,4761,District,3094650,650.0,80.966394,77.072464,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type